In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re

from sklearn import linear_model
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [3]:
df.shape

(13320, 9)

## Data Preprocessing 

Lets start by Data Cleaning. We will remove features to simplify the model

In [4]:
grouped_df = df.groupby(['area_type'])['price'].agg(min_price='min', max_price='max').reset_index()

print(grouped_df)

              area_type  min_price  max_price
0        Built-up  Area       9.00     2050.0
1          Carpet  Area      13.00      500.0
2            Plot  Area       8.44     3600.0
3  Super built-up  Area       8.00     2912.0


 Since the min and max range is quite far we can drop the area_type column

In [5]:
df.drop(['area_type'], axis=1, inplace=True)
df.head()

,availability,location,size,society,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [6]:
# Group by 'are_type' and calculate the list of prices and the mean price
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')
grouped_df = df.groupby(['society','total_sqft'])['price'].agg(min_price='min', max_price='max', average_price='mean').reset_index()

filtered_grouped_df = grouped_df[(grouped_df['max_price'] - grouped_df['min_price']) > 10]

print(filtered_grouped_df)


      society total_sqft  min_price  max_price  average_price
57    AWharde       3198      200.0     250.00     225.000000
99    Adeatlm       2180      170.0     240.00     196.666667
100   Adeatlm       2275      175.0     187.00     181.800000
106   Adeatlm       4000      578.0     600.00     589.000000
107   Adeatlm       5400      700.0     750.00     725.000000
...       ...        ...        ...        ...            ...
5548  Vaniai        1045       58.0      76.77      70.513333
5561  Vaodssh       3600      245.0     260.00     252.500000
5572  Vapasrk       3035      220.0     271.00     245.500000
5602  Veales        1250       72.0      85.00      78.500000
5707  Wiowsri        950       40.0      57.00      47.333333

[341 rows x 5 columns]


For the same society with same total_sqft the prices vary too much. Hence, we can drop society. Also, since we have location we can afford to get rid of society.

In [7]:
df.drop(['society'], axis=1, inplace=True)
df.head()

,availability,location,size,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,1200,2.0,1.0,51.00


In [8]:
# Group by 'are_type' and calculate the list of prices and the mean price
grouped_df = df.groupby(['location','availability'])['price'].agg(prices=list).reset_index()
grouped_df['prices_len'] = grouped_df['prices'].apply(lambda x: len(x))
filtered_grouped_df = grouped_df[(grouped_df['availability'] ) != 'Ready To Move']
# filtered_grouped_df = filtered_grouped_df[(filtered_grouped_df['prices_len'] ) > 1]
print(filtered_grouped_df)


                     location availability              prices  prices_len
3                 Bhoganhalli       19-Dec             [22.89]           1
5         Devarachikkanahalli       18-May              [44.0]           1
9                Rachenahalli       18-May              [19.8]           1
10               Rachenahalli       20-Aug              [28.0]           1
12                Thanisandra       18-Nov             [25.25]           1
...                       ...          ...                 ...         ...
2359             manyata park       18-Nov      [84.83, 61.99]           2
2360        manyata tech park       18-Apr              [56.0]           1
2361  mvj engineering college       18-Jul             [56.53]           1
2372                 tc.palya       18-Apr  [62.0, 55.0, 78.0]           3
2376               whitefiled       20-Dec             [32.73]           1

[1174 rows x 4 columns]


availablity has effect on the final price hence we will keep it

### Handling Missing Data

In [9]:
df.isnull().sum()

availability      0
location          1
size             16
total_sqft        0
bath             73
balcony         609
price             0
dtype: int64

In [10]:
df = df.dropna()

In [11]:
df.isnull().sum()

availability    0
location        0
size            0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64

In [12]:
# rows_with_na = df[df['size'].isna()]
# rows_with_na['indexes'] = df[df['size'].isna()].index
# merged_df = pd.merge(rows_with_na, df, on=['location', 'total_sqft'], suffixes=('_left', '_right'))

# # Fill NaN values in 'size' column with either 'size_left' or 'size_right'
# merged_df['size'] = merged_df['size_left'].fillna(merged_df['size_right'])
# merged_df['bath'] = merged_df['bath_left'].fillna(merged_df['bath_right'])
# merged_df['balcony'] = merged_df['balcony_left'].fillna(merged_df['balcony_right'])

# # Drop unnecessary columns
# suffixes_to_drop = ['_left', '_right']
# merged_df = merged_df.drop([col + suffix for col in ['size', 'bath', 'balcony','availability','price'] for suffix in suffixes_to_drop], axis=1)
# print(merged_df)

# # Display the merged DataFrame with filled NaN values
# for index, i in merged_df.iterrows():
#     print(i)
#     print(df[(df['location']==i.location)  & ( (df['total_sqft'] > i.total_sqft-100) & (df['total_sqft'] < i.total_sqft+100))])
# # df[(df['location']=='IVC Road')  & ( (df['total_sqft'] > 3800) & (df['total_sqft'] < 3900))]
# df.loc[merged_df.indexes.tolist(), ['size', 'bath', 'balcony']] = merged_df[['size', 'bath', 'balcony']].values

# df.head()

### Data Transformation

Since the total_sqft column has str values we need to transform them into floating point numbers

In [13]:
def convert_sqft_to_num(x):
    if pd.isna(x):
        return np.nan
    if '-' in x:
        parts = x.split('-')
        return (float(parts[0]) + float(parts[1])) / 2
    try:
        return float(x)
    except ValueError:
        y = re.match(r'\d+', x)
        y = float(y.group())
        if 'Sq. Meter' in x:
            return y * 10.7639
        if 'Acres' in x:
            return y * 43560
        if 'Sq. Yards' in x:
            return y * 9
        if 'Guntha' in x:
            return y * 1089
        if 'Cents' in x:
            return y * 435.6
        if 'Grounds' in x:
            return y * 2400
        if 'Perch' in x:
            return y * 272.25
        return np.nan

# Apply the function to the total_sqft column
df['total_sqft'] = df['total_sqft'].apply(convert_sqft_to_num)

# Display the DataFrame
df.head()

,availability,location,size,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00


### Handling Categorical Data

We need to change the size from categorical to numerical 

In [14]:
df['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '3 Bedroom', '1 BHK', '1 RK',
       '4 BHK', '1 Bedroom', '2 Bedroom', '6 Bedroom', '8 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '11 Bedroom', '43 Bedroom',
       '14 BHK', '8 BHK', '12 Bedroom', '10 Bedroom', '13 BHK'],
      dtype=object)

In [15]:
df['bhk'] = df['size'].apply(lambda x: int(x.split(' ')[0]))
df.drop(['size'], axis=1, inplace=True)
df.head()

,availability,location,total_sqft,bath,balcony,price,bhk
0,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2
1,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4
2,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3
3,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3
4,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2


In [16]:
df.isnull().sum()

availability    0
location        0
total_sqft      0
bath            0
balcony         0
price           0
bhk             0
dtype: int64

### Feature Engineering 

Dimentionality Reduction - Since there are a lot of diff locations lets group some of them 

In [17]:
print('Unique Locations Before changing case = ', len(df['location'].unique()))
df['location'] = df['location'].str.lower()
print('Unique Locations After changing case = ', len(df['location'].unique()))
df_location_stats = df.groupby(['location'])['location'].agg('count')
df_location_stats.sort_values(ascending=False)

Unique Locations Before changing case =  1265
Unique Locations After changing case =  1256


location
whitefield         514
sarjapur  road     372
electronic city    300
kanakpura road     261
thanisandra        231
                  ... 
junnasandra          1
k r c kothanur       1
kadarenahalli        1
kadubisnahalli       1
zuzuvadi             1
Name: location, Length: 1256, dtype: int64

In [18]:
df_locations_less_than_10 = df_location_stats[df_location_stats<=10]
print('Locations to be grouped = ',len(df_locations_less_than_10))

df['location']=df['location'].apply(lambda x: 'other' if x in df_locations_less_than_10 else x)
print('Unique Locations finally =', len(df['location'].unique()))

Locations to be grouped =  1019
Unique Locations finally = 238


Lets make a new column for price per sqft

In [19]:
df['price_per_sqft']=df['price']*100000/df['total_sqft']
df.head()

,availability,location,total_sqft,bath,balcony,price,bhk,price_per_sqft
0,19-Dec,electronic city phase ii,1056.0,2.0,1.0,39.07,2,3699.810606
1,Ready To Move,chikka tirupathi,2600.0,5.0,3.0,120.00,4,4615.384615
2,Ready To Move,uttarahalli,1440.0,2.0,3.0,62.00,3,4305.555556
3,Ready To Move,lingadheeranahalli,1521.0,3.0,1.0,95.00,3,6245.890861
4,Ready To Move,kothanur,1200.0,2.0,1.0,51.00,2,4250.000000


### Outlier Removal

We can assume that a bedroom is of 200sqft at the very least, so we should remove those entries where the room sizes are too small

In [20]:
print('Total rows =',df.shape)
print('Outliers to be removed =',df[df['total_sqft']/df['bhk']<200].shape)

df = df[~(df['total_sqft']/df['bhk']<200)]
print('Total rows =',df.shape)

Total rows = (12710, 8)
Outliers to be removed = (284, 8)
Total rows = (12426, 8)
